In [2]:
 import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [22]:
#VIDEO FEED
cam =  cv2.VideoCapture(0)
#mediapipe instance
with mp_pose.Pose(min_detection_confidence = 0.5,min_tracking_confidence =0.4) as pose:
    while cam.isOpened() :
        _,frame = cam.read()
        #Detect and render

        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detection
        results = pose.process(image)

        #Recolor back to bgr
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        #Render
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                )

        cv2.imshow("Mediapipe feed",image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cam.release
    cv2.destroyAllWindows()

Sample code for Frog Stand : 


In [28]:
import cv2
import mediapipe as mp
import math

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize Video Capture
cam = cv2.VideoCapture(0)

def calculate_angle(p1, p2, p3):
    """ Calculate angle between three points. """
    angle = math.degrees(
        math.atan2(p3[1] - p2[1], p3[0] - p2[0]) -
        math.atan2(p1[1] - p2[1], p1[0] - p2[0])
    )
    return abs(angle)

def is_frog_stand(angles, distances):
    """ Determine if the posture meets frog stand criteria. """
    # Define acceptable angle ranges and distance thresholds
    elbow_angle_threshold = 90  # degrees
    knee_to_elbow_distance_threshold = 0.1  # normalized distance (relative to image size)
    torso_horizontal_threshold = 10  # degrees (allow some deviation)

    elbow_angle_right, elbow_angle_RIGHT = angles
    knee_to_elbow_distances = distances

    if (elbow_angle_right >= elbow_angle_threshold and
        elbow_angle_RIGHT >= elbow_angle_threshold and
        knee_to_elbow_distances['right'] < knee_to_elbow_distance_threshold and
        knee_to_elbow_distances['RIGHT'] < knee_to_elbow_distance_threshold):
        return True
    return False

while cam.isOpened():
    ret, frame = cam.read()
    if not ret:
        break

    # Process the frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        # Get keypoints
        WR = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        WL = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        ER = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        EL = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        KR = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
        KL = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
        SHR = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        SHL = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

        # Calculate angles
        elbow_knee_angle_right = calculate_angle(
            (SHR.x, SHR.y), (ER.x, ER.y), (KR.x, KR.y)
        )
        elbow_knee_angle_left = calculate_angle(
            (SHL.x, SHL.y), (EL.x, EL.y), (KL.x, KL.y)
        )

        # Calculate distances between knees and elbows
        def euclidean_distance(p1, p2):
            return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

        knee_to_elbow_dist_right = euclidean_distance((KR.x, KR.y), (ER.x, ER.y))
        knee_to_elbow_dist_left = euclidean_distance((KL.x, KL.y), (EL.x, EL.y))

        distances = {
            'right': knee_to_elbow_dist_right,
            'left': knee_to_elbow_dist_left
        }

        angles = (elbow_knee_angle_right, elbow_knee_angle_left)

        # Determine if the posture is a correct frog stand
        if is_frog_stand(angles, distances):
            posture_status = "Frog Stand Correct"
            color = (0, 255, 0)  # Green for correct posture
        else:
            posture_status = "Incorrect Posture"
            color = (0, 0, 255)  # Red for incorrect posture

        # Display results
        cv2.putText(frame, posture_status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow("Frog Stand Detection", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cam.release()
cv2.destroyAllWindows()

In [48]:
#VIDEO FEED
cam =  cv2.VideoCapture(0)
#mediapipe instance
with mp_pose.Pose(min_detection_confidence = 0.5,min_tracking_confidence =0.4) as pose:
    while cam.isOpened() :
        _,frame = cam.read()
        #Detect and render

        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detection
        results = pose.process(image)

        #Recolor back to bgr
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Extract landmakrs
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass



        #Render
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                )

        cv2.imshow("Mediapipe feed",image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cam.release
    cv2.destroyAllWindows()

[x: 0.5374162793159485
y: 0.5845587253570557
z: -1.4989632368087769
visibility: 0.9959070682525635
, x: 0.5706034898757935
y: 0.5158278346061707
z: -1.4945470094680786
visibility: 0.9833266139030457
, x: 0.589247465133667
y: 0.5155249238014221
z: -1.4943203926086426
visibility: 0.9790071249008179
, x: 0.6069096326828003
y: 0.5153518319129944
z: -1.4946728944778442
visibility: 0.9810284972190857
, x: 0.5150024890899658
y: 0.5124331712722778
z: -1.4855575561523438
visibility: 0.9872451424598694
, x: 0.49712762236595154
y: 0.5118120908737183
z: -1.4856548309326172
visibility: 0.9839163422584534
, x: 0.48216065764427185
y: 0.5125719904899597
z: -1.4858026504516602
visibility: 0.986210286617279
, x: 0.6361824870109558
y: 0.5265071392059326
z: -1.2568742036819458
visibility: 0.9760810136795044
, x: 0.45174717903137207
y: 0.5204833745956421
z: -1.184749722480774
visibility: 0.9888312816619873
, x: 0.5626611709594727
y: 0.6526440382003784
z: -1.393160343170166
visibility: 0.9948115348815918
, 

In [50]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value] # Example for left shoulder

x: 0.7553737163543701
y: 0.8136106133460999
z: -0.2778673470020294
visibility: 0.9997331500053406

Calculating angle

In [40]:
def calculate_angle(a,b,c):
    a = np.array(a) #First
    b = np.array(b) #Mid
    c = np.array(c) #End

    radians = np.arctan2(c[1]-b[1],c[0]-b[0] - np.arctan2(a[1]-b[1],a[0]-b[0]))
    angle = np.abs(radians*180/np.pi)
    if angle > 180.0:
        angel = 360-angle

    return angle

In [54]:
shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

In [55]:
shoulder,elbow,wrist

([0.35044342279434204, 0.7333894371986389],
 [0.06844791024923325, 0.7349022626876831],
 [0.04517286270856857, 0.29939115047454834])

In [56]:
calculate_angle(shoulder,elbow,wrist)

92.35496056291963

In [58]:
np.multiply(elbow,[640,480])

array([ 43.80666256, 352.75308609])

In [66]:
#VIDEO FEED
cam =  cv2.VideoCapture(0)
#mediapipe instance
with mp_pose.Pose(min_detection_confidence = 0.5,min_tracking_confidence =0.4) as pose:
    while cam.isOpened() :
        _,frame = cam.read()
        #Detect and render

        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detection
        results = pose.process(image)

        #Recolor back to bgr
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Extract landmakrs
        try:
            landmarks = results.pose_landmarks.landmark
            #Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            #Calculate angle
            angle = calculate_angle(shoulder,elbow,wrist)
            angle = int(angle)

            #Visualize
            cv2.putText(image,str(angle),
                        tuple(np.multiply(elbow,[640,480]).astype(int)),
                              cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
            print(landmarks)
        except:
            pass



        #Render
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                )

        cv2.imshow("Mediapipe feed",image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cam.release
    cv2.destroyAllWindows()

[x: 0.4374854564666748
y: 0.461434006690979
z: -0.7199997305870056
visibility: 0.9980214834213257
, x: 0.45761650800704956
y: 0.4280986785888672
z: -0.6937372088432312
visibility: 0.9947869777679443
, x: 0.4689759612083435
y: 0.4299154281616211
z: -0.6937353610992432
visibility: 0.995317816734314
, x: 0.4801848530769348
y: 0.43310248851776123
z: -0.6937463283538818
visibility: 0.9942585229873657
, x: 0.425914466381073
y: 0.4268539547920227
z: -0.6948904395103455
visibility: 0.9965112805366516
, x: 0.4153175950050354
y: 0.42764586210250854
z: -0.6950563192367554
visibility: 0.9972538352012634
, x: 0.40529316663742065
y: 0.42945653200149536
z: -0.695475697517395
visibility: 0.9974372386932373
, x: 0.5016513466835022
y: 0.46368110179901123
z: -0.5084452033042908
visibility: 0.9955992698669434
, x: 0.38883474469184875
y: 0.45324385166168213
z: -0.5032296180725098
visibility: 0.9983003735542297
, x: 0.45611143112182617
y: 0.5098193287849426
z: -0.6486521363258362
visibility: 0.9993212223052

In [77]:
#VIDEO FEED
cam =  cv2.VideoCapture(0)
# Curl counter variables
counter = 0 
stage = None
#mediapipe instance
with mp_pose.Pose(min_detection_confidence = 0.5,min_tracking_confidence =0.4) as pose:
    while cam.isOpened() :
        _,frame = cam.read()
        #Detect and render

        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Make detection
        results = pose.process(image)

        #Recolor back to bgr
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #Extract landmakrs
        try:
            landmarks = results.pose_landmarks.landmark
            #Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            #Calculate angle
            angle = calculate_angle(shoulder,elbow,wrist)
            angle = int(angle)

            #Visualize
            cv2.putText(image,str(angle),
                        tuple(np.multiply(elbow,[640,480]).astype(int)),
                              cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
             
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
        except:
            pass


 
        # Setup status box
        cv2.rectangle(image, (500,0), (725,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (515,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (510,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (565,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (560,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
        
        #Render
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                )

        cv2.imshow("Mediapipe feed",image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cam.release
    cv2.destroyAllWindows()

1
2
